## T-REX wrapper

First we import functions.

In [ ]:
import numpy as np
from tqdm import tqdm
from newick import read
from datasets.loading import load_data
import os
import ipdb

os.environ['DATAPATH']="./data"
os.environ['SAVEPATH']="./embeddings"

def sim2metric(S):
    """
    Input S is a n-by-n np array, representing similarities.
    Output D turn S into a metric.
    It should satisfy:
    1) Sij = 1 iff Dij = 0
    2) Sij = -1 iff Dij = max(D)
    3) basic metric properties (triangle ineq...etc)
    
    Now we choose D = 1-S
    """
    return 1-S

def save_dist2phylip(D,path):
    n = D.shape[0]
    f = open(path, "w")
    f.write(str(n)+'\n')
    for i in tqdm(range(n)):
        f.write(str(i)+' ')
        np.savetxt(f,D[i],fmt='%.4f',newline=" ")
        f.write('\n')
    f.close()
    print("Matrix saved")

def newick2dist(trees,n):
    # This convert a tree from newick format to distance matrix (n-by-n)
    # Note that all internal nodes are named "None"
    # All leaves node have name in np.arange(n)
    
    D = np.zeros((n,n))
    D = Fill_D_from_tree(trees[0],D)
    return D

def Fill_D_from_tree(subtree,D):
    All_leaves = np.arange(D.shape[0])    

    num_children = len(subtree.descendants)
    for i in range(num_children):
        sub_leaves = np.array(subtree.descendants[i].get_leaf_names()).astype(int)
        # We simply set the negative branch weights to zero following the common practice
        if subtree.descendants[i].length>0:
            D[np.ix_(sub_leaves,np.setdiff1d(All_leaves,sub_leaves))] += subtree.descendants[i].length
            D[np.ix_(np.setdiff1d(All_leaves,sub_leaves),sub_leaves)] += subtree.descendants[i].length
    
    
        if subtree.descendants[i].name == None:
            D = Fill_D_from_tree(subtree.descendants[i],D)
    
    return D

In [ ]:
from scipy.sparse import csr_matrix, lil_matrix
from scipy.sparse.csgraph import shortest_path
import newick

def newick2W(trees,n):
    # This convert a tree from newick format to distance csr matrix, including interior nodes ((2n-1)-by-(2n-1))
    # Note that all internal nodes are named "None"
    # All leaves node have name in np.arange(n)
    
    W = csr_matrix(((2*n-1),(2*n-1)))
    assert len(trees[0].descendants) == 2 # An internal/root node should always have 2 children!
    
    W,_,_ = Fill_W_from_tree(trees[0],W,n)
    return W

def Fill_W_from_tree(subtree,W,cur_idx):   
    # Check if we visit current node
    if subtree.name == None:
        # Not visit yet, give id
        subtree.name = str(cur_idx)
        cur_idx += 1
    # Now add all adjacent relation to W
    if not subtree.is_leaf:
        Flag = False
        for i in range(len(subtree.descendants)):
            if subtree.descendants[i].name == None:
                subtree.descendants[i].name = str(cur_idx)
                cur_idx += 1
                Flag = True

            W[int(subtree.descendants[i].name),int(subtree.name)] = subtree.descendants[i].length
            W[int(subtree.name),int(subtree.descendants[i].name)] = subtree.descendants[i].length

            if Flag:
                W,subtree.descendants[i],cur_idx = Fill_W_from_tree(subtree.descendants[i],W,cur_idx)
    
    return W,subtree,cur_idx

def generate_rand_TreeMetric_V2(N,option='unweight',noise_ratio=0.0):
    # Generate (weighted) random binary tree in newick format.
    T = generate_tree_newick(np.arange(N),option=option)
    # Make it into a tree data structure in python
    T = newick.loads(T)
    # Get the distance matrix for leaves in T
    W = newick2W(T,N)
    
    # Add noise. We randomly add noise_ratio*(2n-2) fake connections between nodes (including internal nodes).
    m_Noise = int(noise_ratio*(2*N-2))
    for _ in range(m_Noise):
        idx0,idx1 = np.random.choice(N,size=(2),replace=False)
        if option == 'unweight':
            W[idx0,idx1] = 1.0
            W[idx1,idx0] = 1.0
        else:
            W[idx0,idx1] = np.random.random()
            W[idx1,idx0] = np.random.random()
    
    # Now construct the shortest path based on this noisy observation
    Noisy_D = get_leaves_Dmatrix(W,N)
    
    return Noisy_D

def generate_rand_TreeMetric(N,option='unweight',noise_scale=1.0,p=2):
    # Generate (weighted) random binary tree in newick format.
    T = generate_tree_newick(np.arange(N),option='unweight')
    # Make it into a tree data structure in python
    T = newick.loads(T)
    # Get the distance matrix for leaves in T
    D = newick2dist(T,N)
    
    # Add noise (noise_scale*uniform([0,1])) to each leaf distance. 
    Noise = np.around(np.triu(noise_scale*np.random.random(size=D.shape),1),5)
    Noise = Noise + Noise.transpose()
    Noisy_D = D + Noise
    Noise_norm = (np.abs(Noise)**p).sum().sum()**(1/p)
    print('Noise_norm:{0:4f}'.format(Noise_norm))
    
    return Noisy_D

def generate_tree_newick(L,option='unweight'):
    # base case
    if len(L) == 1: 
        return str(L[0])
    else:
        perm_L = np.random.permutation(L)
        split = np.random.randint(1,len(L))
        left = perm_L[:split]
        right = perm_L[split:]
    # recursion
    if option == 'unweight':
        return str((generate_tree_newick(left)+':1.0', generate_tree_newick(right)+':1.0')).replace(' ','').replace("'","")
    else:
        left_weight = ':{0:.5f}'.format(np.random.rand())
        right_weight = ':{0:.5f}'.format(np.random.rand())
        return str((generate_tree_newick(left,option=option)+left_weight, generate_tree_newick(right,option=option)+right_weight)).replace(' ','').replace("'","")

def newick2dist(trees,n):
    # This convert a tree from newick format to distance matrix (n-by-n)
    # Note that all internal nodes are named "None"
    # All leaves node have name in np.arange(n)
    
    D = np.zeros((n,n))
    assert len(trees[0].descendants) == 2 # An internal/root node should always have 2 children!
    
    
    D = Fill_D_from_tree(trees[0],D)
    return D

def Fill_D_from_tree(subtree,D):
    All_leaves = np.arange(D.shape[0])    

    num_children = len(subtree.descendants)
    for i in range(num_children):
        sub_leaves = np.array(subtree.descendants[i].get_leaf_names()).astype(int)
        # We simply set the negative branch weights to zero following the common practice
        if subtree.descendants[i].length>0:
            D[np.ix_(sub_leaves,np.setdiff1d(All_leaves,sub_leaves))] += subtree.descendants[i].length
            D[np.ix_(np.setdiff1d(All_leaves,sub_leaves),sub_leaves)] += subtree.descendants[i].length
    
    
        if subtree.descendants[i].name == None:
            D = Fill_D_from_tree(subtree.descendants[i],D)
    
    return D

def generate_rand_Emetric(num_nodes,E_rank=2):
    X = np.random.normal(size=(num_nodes,E_rank))
    X = X/np.linalg.norm(X,axis=1,keepdims=True)*np.random.rand(num_nodes,1)
    D_metric = np.zeros((num_nodes,num_nodes))
    all_pairs = generate_all_pairs(num_nodes)
    D_metric[all_pairs[:,0],all_pairs[:,1]] = np.linalg.norm(X[all_pairs[:,0],:]-X[all_pairs[:,1],:],axis=1)
    D_metric[all_pairs[:,1],all_pairs[:,0]] = D_metric[all_pairs[:,0],all_pairs[:,1]]
    return D_metric

def generate_Emetric(x):
    D_metric = np.zeros((x.shape[0],x.shape[0]))
    all_pairs = generate_all_pairs(x.shape[0])
    D_metric[all_pairs[:,0],all_pairs[:,1]] = np.linalg.norm(x[all_pairs[:,0],:]-x[all_pairs[:,1],:],axis=1)
    D_metric[all_pairs[:,1],all_pairs[:,0]] = D_metric[all_pairs[:,0],all_pairs[:,1]]
    # Round first to prevent numerical issue...
    D_metric = np.round(D_metric,5)
    return D_metric

def sim2metric(S):
    """
    Input S is a n-by-n np array, representing similarities.
    Output D turn S into a metric.
    It should satisfy:
    1) Sij = 1 iff Dij = 0
    2) Sij = -1 iff Dij = max(D)
    3) basic metric properties (triangle ineq...etc)
    
    Now we choose D = 1-S
    """
    return 1-S

def generate_tree(L):
    # base case
    if len(L) == 1: 
        return L[0]
    else:
        perm_L = np.random.permutation(L)
        split = np.random.randint(1,len(L))
        left = perm_L[:split]
        right = perm_L[split:]
    # recursion
    return (generate_tree(left), generate_tree(right))

def plot_rand_binary_tree(brt,n,plot=True,normalization=False,d_mtx=None):
    w_star = None
    A = np.zeros((2*n-1,2*n-1))
    # Note that we keep the leave node id the same!!!
    # Thus the internal node id start at n.
    # For clarity, we set node id n to be the root node
    A,next_idx = Fill_adj(brt,n,A)
    
    assert next_idx == 2*n-1
    
    if normalization:
        # Apply Puoya's normalized weight
        A_T=np.zeros((n*(n-1)/2,2*n-2))
        # First construct an edge name dictionary
        w_dict = -np.ones((2*n-1,2*n-1))
        w_dict,next_idx,cur_w_idx = Fill_w_dict(brt,n,w_dict)
        assert next_idx == 2*n-1
        assert cur_w_idx == 2*n-2
        
        # Now go through all shortest path for leaves and get d_vec
        G = nx.from_numpy_matrix(np.matrix(A))
        All_path = nx.shortest_path(G)
        count = 0
        d_vec = np.zeros(n*(n-1)/2)
        for i in range(n):
            for j in range(i+1,n):
                d_vec[count] = d_mtx[i,j]
                path = All_path[i][j]
                for k in range(len(path)-1):
                    A_T[count,int(w_dict[path[k],path[k+1]])] = 1
                count += 1
        
        # optimal weight
        w_star = np.matmul(np.matmul(np.linalg.pinv(np.matmul(A_T.transpose(),A_T)),A_T.transpose()),d_vec)
        # modify adj matrix A
        for i in range(2*n-1):
            for j in range(i+1,2*n-1):
                if A[i,j] == 1:
                    A[i,j] = w_star[int(w_dict[i,j])]
                    A[j,i] = w_star[int(w_dict[i,j])]
    
    # Compute d, the shortest path for leave nodes
    G = nx.from_numpy_matrix(np.matrix(A))
    d = nx.algorithms.shortest_paths.dense.floyd_warshall_numpy(G)
    d = d[:n]
    d = d[:,:n]
    
    if plot:
        plot_Tree_metric(A,n,n)
    
    return d,w_star

def Fill_adj(brt,cur_idx,A):
    left = brt[0]
    if isinstance(left,np.int64):
        A[cur_idx,left] = 1
        A[left,cur_idx] = 1
        next_idx = cur_idx+1
    else:
        cur_idx_left = cur_idx+1
        A[cur_idx,cur_idx_left] = 1
        A[cur_idx_left,cur_idx] = 1
        A,next_idx = Fill_adj(left,cur_idx_left,A)
    
    right = brt[1]
    if isinstance(right,np.int64):
        A[cur_idx,right] = 1
        A[right,cur_idx] = 1
    else:
        cur_idx_right = next_idx
        A[cur_idx,cur_idx_right] = 1
        A[cur_idx_right,cur_idx] = 1
        A,next_idx = Fill_adj(right,cur_idx_right,A)

    return A,next_idx
    
def Fill_w_dict(brt,cur_idx,w_dict,cur_w_idx=0):
    left = brt[0]
    if isinstance(left,np.int64):
        w_dict[cur_idx,left] = cur_w_idx
        w_dict[left,cur_idx] = cur_w_idx
        next_idx = cur_idx+1
        cur_w_idx += 1
    else:
        cur_idx_left = cur_idx+1
        w_dict[cur_idx,cur_idx_left] = cur_w_idx
        w_dict[cur_idx_left,cur_idx] = cur_w_idx
        cur_w_idx += 1
        w_dict,next_idx,cur_w_idx = Fill_w_dict(left,cur_idx_left,w_dict,cur_w_idx)
    
    right = brt[1]
    if isinstance(right,np.int64):
        w_dict[cur_idx,right] = cur_w_idx
        w_dict[right,cur_idx] = cur_w_idx
        cur_w_idx += 1
    else:
        cur_idx_right = next_idx
        w_dict[cur_idx,cur_idx_right] = cur_w_idx
        w_dict[cur_idx_right,cur_idx] = cur_w_idx
        cur_w_idx += 1
        w_dict,next_idx,cur_w_idx = Fill_w_dict(right,cur_idx_right,w_dict,cur_w_idx)

    return w_dict,next_idx,cur_w_idx

def plot_Tree_metric(W,N_leaves,root_id=-1,Return_metric=False,plot=True,option='scipy'):
    
    print('Start computing shortest path')
    
    # First to remove potential zero rows
    W = W[~np.all(W == 0, axis=1)]
    W = W[:,~np.all(W == 0, axis=0)]
    
#     # Replace all negative entries to 0 (if any)
#     W[W<0] = 0.0
    
#     # Rounding for better vitualization
#     W = np.around(W,3)
    
    G = nx.from_numpy_matrix(np.matrix(W))
    
    if plot:
        # Use spring_layout to handle positioning of graph
        layout = nx.kamada_kawai_layout(G)

        # # Use a list for node_sizes
        # sizes = [1000,400,200]

        # # Use a list for node colours
        # # Here we use different color to distiguish leave nodes and steiner nodes
        # # By default, we use 'tab:blue' for leave nodes and 'tab:red' for steiner nodes
        N_all = W.shape[0]
        color_map = []
        for _ in range(N_leaves):
            color_map.append('tab:blue')
        for _ in range(N_all-N_leaves):
            color_map.append('tab:red')

        if root_id>-1:
            color_map[root_id] = 'tab:green'

        # Draw the graph using the layout - with_labels=True if you want node labels.
        nx.draw(G, layout, with_labels=True, node_color= color_map)

        # Get weights of each edge and assign to labels
        labels = nx.get_edge_attributes(G, "weight")

        # Draw edge labels using layout and list of labels
        nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=labels)
    
    # Also compute the shortest distance matrix for leaves
    if Return_metric:
        if option == 'networkx':
            n = N_leaves
            d = nx.algorithms.shortest_paths.dense.floyd_warshall_numpy(G)
            d = d[:n]
            d = d[:,:n]
            return np.array(d)
        elif option == 'scipy':
            n = N_leaves
            graph = csr_matrix(W) 
            d = shortest_path(csgraph=graph,directed=False)
            d = d[:n]
            d = d[:,:n]
            return d
    else:
        return None
    
def is_ultrametric(M):
    # First check if all entries are non-negative!
    if M[M<0].any():
        print("Input metric has negative distance!")
        return False
    n = M.shape[0]
    
    for i in range(n):
        for j in range(i):
            for k in range(j):
                if M[i,j] > max(M[i,k],M[j,k]):
                    return False
                if M[i,k] > max(M[i,j],M[j,k]):
                    return False 
                if M[j,k] > max(M[i,j],M[i,k]):
                    return False
    
    return True

def is_treemetric(M):
    # First check if all entries are non-negative!
    if M[M<0].any():
        print("Input metric has negative distance!")
        return False
    
    # Also compute Gromov hyperbolicity (delta)
    n = M.shape[0]
    Delta = 0
    Flag = True
    for i in tqdm(range(n)):
        for j in range(i):
            for k in range(j):
                for m in range(k):
                    ARRAY = np.array([i,j,k,m])
                    for elmt in list(itertools.permutations(ARRAY)):
                        if M[elmt[0],elmt[1]]+M[elmt[2],elmt[3]] > max(M[elmt[0],elmt[2]]+M[elmt[1],elmt[3]],M[elmt[0],elmt[3]]+M[elmt[1],elmt[2]]):
                            diff = M[elmt[0],elmt[1]]+M[elmt[2],elmt[3]] - max(M[elmt[0],elmt[2]]+M[elmt[1],elmt[3]],M[elmt[0],elmt[3]]+M[elmt[1],elmt[2]])
                            if diff > Delta:
                                Delta = diff
                            Flag = False
                            
                    
    return Flag, Delta

def get_leaves_Dmatrix(W,N_leaves):
    # Input W is a csr_matrix.
    n = N_leaves 
    d = shortest_path(csgraph=W,directed=False)
    d = d[:n]
    d = d[:,:n]
    return d

def Linkage2dist(Z):
    # First, build a dictionary for mapping all nodes to actual cluters
    n = Z.shape[0]+1
    # Include all leaves as singleton first
    W_Size = int(max(Z[:,0].max(),Z[:,1].max()))+2
    W = csr_matrix((W_Size,W_Size))    

    for i in range(Z.shape[0]):
        idx0 = Z[i,0]
        idx1 = Z[i,1]
        length = Z[i,2]
        pid = i+n
        W[idx0,pid] = length
        W[idx1,pid] = length
        W[pid,idx0] = length
        W[pid,idx1] = length

    d = shortest_path(csgraph=W,directed=False)
    d = d[:n]
    d = d[:,:n]
    return d

## Save the dissimilarity matrix for real world datasets

In [ ]:
DATASET = 'zoo'
CURV = '100' # set it to be the curvature you use for using HyperAid.
SEED = '0' # Change if necessary.
x, y_true, similarities = load_data(DATASET)
D_metric = sim2metric(similarities)
np.save('./embeddings/{}/seed_{}_curv_{}/D_target_{}_{}.npy'.format(DATASET,SEED,CURV,DATASET,SEED),D_metric) #

## Save the raw input metric and trained hyperbolic metric in phylip format. 

In [ ]:
"""
Note that D_metric_$dataset_$seed.npy is D_hyp
D_hyp_$dataset_$seed.phylip is its phylip format
D_metric_$dataset_$seed.phylip is the taget metric in phylip format
"""

x, y_true, similarities = load_data(DATASET)
D_metric = sim2metric(similarities)

D_hyp_path = './embeddings/{}/seed_{}_curv_{}/D_metric_segmentation_{}.npy'.format(DATASET,SEED,CURV,DATASET,SEED)
D_hyp = np.load(D_hyp_path)

D_metric_phylo_path = './embeddings/{}/seed_{}_curv_{}/D_metric.phylip'.format(DATASET,SEED,CURV)
D_hyp_phylo_path = './embeddings/{}/seed_{}_curv_{}/D_hyp.phylip'.format(DATASET,SEED,CURV)
save_dist2phylip(D_metric,D_metric_phylo_path)
save_dist2phylip(D_hyp,D_hyp_phylo_path)

## Applying T-REX [website](http://www.trex.uqam.ca/index.php?action=home)

1) First click "NJ" on the left panel.
2) Choose "Distance matrix".
3) Click "File" then upload the .phylip file.
4) Choose the desired tree reconstruction method, NJ or UNJ.
5) After redirecting to the result page, click "Fitting statistics".
6) Copy the numbers right after "TREE METRIC (ADDITIVE DISTANCE) MATRIX (AD)", save it to the same folder with name `D_hyp_{DEC_method}.phylip` if the input file is `D_hyp.phylip` and `D_metric_{}.phylip` for the other case.

In [ ]:
DEC = 'NJ' # choose 'NJ' or 'UNJ'

Hyp_dec_path = './embeddings/{}/seed_{}_curv_{}/D_hyp_{}.phylip'.format(DATASET,SEED,CURV,DEC)
Metric_dec_path = './embeddings/{}/seed_{}_curv_{}/D_metric_{}.phylip'.format(DATASET,SEED,CURV,DEC)

def read_phylip2npy(Hyp_NJ_path):
    f = open(Hyp_NJ_path,'r')
    Array = np.loadtxt(f,)
    Array = Array[:,1:]
#     print(Array)
    f.close()
    return Array

D_metric_dec_output = read_phylip2npy(Metric_dec_path)
loss_metric_dec = ((D_metric_dec_output-D_metric)**2).sum().sum()**(1/2)
D_hyp_dec_output = read_phylip2npy(Hyp_dec_path)
loss_hyp_dec = ((D_hyp_dec_output-D_metric)**2).sum().sum()**(1/2)
print('l2 loss for Direct: ',loss_metric_dec)
print('l2 loss for HyperAid: ',loss_hyp_dec)
print('Gain (%): ',(loss_metric_dec/loss_hyp_dec-1)*100)